# Move tests

In [25]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [20]:
s = Scene(init='cup')
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 2 0], empty
Robot: [3 2 3], opened, rotation: 1, attached: -


### 1. Fake move

In [3]:
s.r.eef_position = s.cup.position

In [4]:
Actions.do(s, ('pick_up','cup'))
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 3 0], empty
Robot: [2 3 0], closed, rotation: 1, attached: cup


In [5]:
s = Scene(init='cup,cup')
s.r.eef_position = s.cup.position
Actions.do(s, ('pick_up', 'cup'))

True

In [6]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 3 0], full
cup1, cup, [2 3 0], full
Robot: [3 3 0], opened, rotation: 0, attached: cup


### 2. Move via actions
- Move actions can be executed independently `Actions.do(s, ('move_xxx', None))`, when set of actions contains the desired move actions

In [9]:
Actions.do(s, ('move_up',None))

True

In [10]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 3 2], full
cup1, cup, [2 3 0], full
Robot: [3 3 2], opened, rotation: 0, attached: cup


### 3. Non-move actions with ignore_location flag
- do not consider eef position

In [12]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [21]:
s = Scene(init='cup')
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 1 0], empty
Robot: [1 3 3], closed, rotation: 0, attached: -


In [22]:
Actions.do(s, ('pick_up', 'cup'))

False

Robot eef and cup are not at one position, but action `pick_up` can be performed when `ignore_location=True`.

In [23]:
Actions.do(s, ('pick_up', 'cup'), ignore_location=True)

True

### 4. Non-move action with handle_location flag
- Program will automatically execute move action sequence and move eef to desired object position
- Move actions are planned and executed from start (eef) to target (object)
- When there is obstacle on the scene, the some move actions are not valid for given eef position, the planner finds the way of valid actions to reach the target

In [26]:
s = Scene(init='cup')
Actions.do(s, ('pick_up', 'cup'), handle_location=True)

Executed move actions: ['move_down', 'move_left', 'move_down', 'move_back', 'move_left', 'move_down']


True

handle_location flag calls function `s.plan_path_to_position(..)` and then `Actions.execute_path_to_position(..)`
Example:

In [51]:
s = Scene(init='cup')
s.r.eef_position

array([1, 3, 3])

In [52]:
move_action_seq = s.plan_path_to_position([0,0,0], Actions)
Actions.execute_path_to_position(s,move_action_seq)

True

In [53]:
move_action_seq

['move_left',
 'move_down',
 'move_left',
 'move_down',
 'move_front',
 'move_left',
 'move_down']

In [54]:
s.r.eef_position

array([0, 0, 0])

In [68]:
Actions.get_possible_move_actions(s)

['move_back', 'move_up', 'move_front', 'move_left']

### Helper functions
- `get_possible_actions`
- `get_possible_actions_bool_array`

In [71]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [72]:
s = Scene(init='cup,cup,drawer,object')

In [73]:
Actions.get_possible_actions(s, ignore_location=True)

[('close', 'drawer'),
 ('pick_up', 'cup'),
 ('pick_up', 'cup1'),
 ('pick_up', 'object')]

In [74]:
Actions.get_possible_actions_bool_array(s, ignore_location=True)

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False,  True, False],
       [ True,  True, False,  True]])

In [75]:
Actions.do(s,('pick_up', 'cup'), ignore_location=True)

True

In [76]:
Actions.get_possible_actions(s, ignore_location=True)

[('put', 'drawer'), ('close', 'drawer')]

In [77]:
Actions.get_possible_actions_bool_array(s, ignore_location=True)

array([[False, False, False, False],
       [False, False, False, False],
       [False, False,  True, False],
       [False, False, False, False],
       [False, False,  True, False],
       [False, False, False, False]])